In [ ]:
using DrWatson
@quickactivate "masterarbeit"

In [ ]:
using BenchmarkTools
using ProgressMeter
using CUDA
using Flux
using LaTeXStrings
using Flux: train!
using GLMakie
using Printf
using Dates
using JLD2
using TOML
using StatsBase # for fit(histogram)

In [ ]:
Makie.inline!(true)
fontsize_theme = Theme(fontsize=35)
set_theme!(fontsize_theme)
wblue = Makie.wong_colors()[1]
worange = Makie.wong_colors()[2]
wgreen = Makie.wong_colors()[3]
wpink = Makie.wong_colors()[4]
wlblue = Makie.wong_colors()[5]
worange = Makie.wong_colors()[6]
wyellow = Makie.wong_colors()[7];

In [ ]:
using Revise

In [ ]:
using masterarbeit

In [ ]:
function jacobian2cl(m::Chain, cm::ChannelMapping, x::T) where {T <: AbstractArray{F}} where F<:Real
    cl1 = m[1]
    sl1 = m[2]
    cl2 = m[3]
    x2 = cl1(x)
    x2s = sl1(x2)
    det1 = abs.(cldet(cl1,  x[cl1.dimA+1:cl1.d,:], cl1.m( x[1:cl1.dimA,:])...))
    det2 = abs.(cldet(cl2, x2s[cl2.dimA+1:cl2.d,:], cl2.m(x2s[1:cl2.dimA,:])...))
    return abs(cmdet(cm)) .* det1 .* det2
end

In [ ]:
function lossf(m::Chain, cm::ChannelMapping, f::Function, x::T) where T<:AbstractArray{F} where F<:Real
    zi = cm(m(x))
    g = 1 ./ jacobian2cl(m, cm, x)
    fracs = abs.(f(zi) .- g) .^F(1.5) ./ g
    return sum(fracs) / size(x,2)
end

In [ ]:
omega = 0.1
dphi = 10.0
f = x -> comptonf(x, omega, dphi)
ytozmap = HypercubeTocθωbar()
dim = 2
dimA = 1;

In [ ]:
batchsize = 4096
N_epochs = 50
bins = 20
learning_rate = 0.01
weight_decay = 4.3E-4
decay = 0.7
optimizer = Adam
;

In [ ]:
function subnet(dimA::Signed, dimB::Signed, bins::Signed, width=8)
    return Chain(
        Split(
            Chain(
                BatchNorm(dimA),
                Dense(dimA => width, relu),
                Dense(width => width, relu),
                Dense(width => dimB*(bins+1))  
                ), 
            Chain(
                BatchNorm(dimA),
                Dense(dimA => width, relu),
                Dense(width => width, relu),
                Dense(width => dimB*bins)
                )
            ) 
        ) |> gpu
end

In [ ]:
model = Flux.f64(Chain(
    CouplingLayer(dim, dimA, bins, subnet),
    masterarbeit.MaskLayer([false, true]),
    CouplingLayer(dim, dimA, bins, subnet),
) |> gpu);

In [ ]:
# first run to compile/test
xtest = CUDA.rand(Float64,dim,batchsize);

In [ ]:
model(xtest);

In [ ]:
f(ytozmap(xtest));

In [ ]:
lossf(model,ytozmap,f,xtest);

In [ ]:
Flux.withgradient(m-> lossf(m,ytozmap,f,xtest), model);

# Training

In [ ]:
losses = Float64[]

In [ ]:
current_learning_rate = learning_rate
for i in 1:3
    println("Training with learning rate  = $(current_learning_rate)")
    losses = train_NN(model, dim, lossf, losses, ytozmap, f, epochs=100, batchsize=batchsize, optimizer=optimizer, learning_rate=current_learning_rate, ftype=Float64)
    current_learning_rate = current_learning_rate * decay
end;

In [ ]:
losses[end]

In [ ]:
fig = Figure(size=(1500,1000))
ax = Axis(fig[1,1], xlabel="epoch", ylabel="loss")
lines!(1:length(losses), losses, linewidth=4, color=wblue, label="loss")
n = 10
lines!(n:length(losses), moving_average(losses, n), linewidth=4, color=worange, label="$n epoch \n moving average")
fig[1,2] = Legend(fig, ax)
save("compton_nis_loss.png", fig)
fig

In [ ]:
samples = sample_NN(model, ytozmap, dim, 10^7, batchsize*4, ftype=Float64);

In [ ]:
xticks = [-1.0, -0.5, 0.0, 0.5, 1.0]
yticks = [0.0, 0.25, 0.5, 0.75, 1.0];

In [ ]:
function makie_samples(samples)
    histo = fit(Histogram, (samples[1,:], samples[2,:]), nbins=200)
    fig = Figure(size=(1000,1000), figure_padding=40)
    ax = Axis(fig[1,1], xticks=xticks, yticks=yticks, xlabel=L"\cos{\theta}", ylabel=L"\overline{\omega'}",
        aspect=1, xlabelsize=50, ylabelsize=50)
    heatmap!(histo.edges[1], histo.edges[2], histo.weights)
    ylims!(0.0,1.0)
    fig
end
fig = makie_samples(samples)
save("compton_nis_samples.png", fig)
fig

In [ ]:
fig = Figure(size=(1000,1000), figure_padding=40)
ax = Axis(fig[1,1], xticks=xticks, yticks=yticks, xlabel=L"\cos{\theta}", ylabel=L"\overline{\omega'}")
xs = LinRange(-1.0, 1.0-eps(), 1001)
ys = LinRange(0.0+eps(), 1.0-eps(), 1001)
zs = [f([x;y])[1] for x in xs, y in ys]
heatmap!(xs, ys, zs)
save("compton_truth.png", fig)
fig

In [ ]:
wi_vegas = load_object("vegas_weights.jld2");

In [ ]:
function weights2cl(m::Chain, cm::ChannelMapping, f::Function, x::T) where {T <: AbstractArray}
    return jacobian2cl(m, cm, x) .* f(cm(m(x)))'
end

function weights2cl_chunked(m, dim, cm, f, N, batchsize)
    if (N%batchsize != 0) 
        x = CUDA.rand(Float64, dim, N%batchsize)
        weights = weights2cl(m, cm, f, x) |> cpu
        inputs = x
        runs = N ÷ batchsize 
    else
        x = CUDA.rand(Float64, dim,   batchsize)
        weights = weights2cl(m, cm, f, x) |> cpu
        inputs = x
        runs = N ÷ batchsize - 1
    end
    for i in 1:runs
        x = CUDA.rand(Float64, dim, batchsize)
        weights = hcat(weights, weights2cl(m, cm, f, x) |> cpu)
        inputs = hcat(inputs, x)
    end
    return weights, inputs
end

wi_m, x_for_wi = weights2cl_chunked(model, dim, ytozmap, f,  10^7, batchsize*4)
wi = wi_m[1,:]
w_avg = mean(wi)
w_max = maximum(wi)
wi_n = wi ./ w_max
w_avg_n = mean(wi_n)
println("mean weight = $(mean(wi))")
println("max weight = $(maximum(wi))")
println("unweighting efficiency = $(mean(wi)/maximum(wi))")

In [ ]:
wi_vegas_filtered = wi_vegas[wi_vegas .< 10.01];

In [ ]:
fig = Figure(size=(1500,1000))
ax = Axis(fig[1,1], ylabel=L"N", xlabel=L"w", yscale=Makie.pseudolog10, yticks=[0, 10^1, 10^3, 10^5, 10^7])
stephist!(wi_vegas ./ integral, linewidth=3, color=wblue, label="VEGAS", bins=100) 
stephist!(wi ./ integral, linewidth=3, color=worange, label="NIS", bins=100) 
fig[1,2] = Legend(fig, ax)

inset_ax = Axis(fig[1, 1], yscale=Makie.pseudolog10, yticks=[0, 10^1, 10^3, 10^5, 10^7], 
    width=Relative(0.65), height=Relative(0.65), halign=0.92, valign=0.92,)
translate!(inset_ax.elements[:background], 0, 0, -10)
stephist!(wi_vegas_filtered ./ integral, linewidth=3, color=wblue, label="VEGAS", bins=50) 
stephist!(wi ./ integral, color=worange, linewidth=3, label="NIS", bins=50) 
xlims!(0.0, 150.0)

save("compton_weights.png", fig)
fig

In [ ]:
x_signal = x_for_wi[:, wi .>= 0.1]
x_noise = x_for_wi[:, wi .< 0.1];

In [ ]:
function sample_chunked(m, dim, cm, x, batchsize)
    res = Array{Float64}(undef, dim, 1)
    runs = size(x,2) ÷ batchsize
    for i in 1:runs
        res = hcat(res, cm(m(x[:,(1+(i-1)*batchsize):(1+i*batchsize)]))|>cpu)
    end
    return res
end

In [ ]:
signal_samples = sample_chunked(model, dim, ytozmap, x_signal, batchsize);

In [ ]:
noise_samples = sample_chunked(model, dim, ytozmap, x_noise, batchsize);

In [ ]:
fig = makie_samples(signal_samples)
save("compton_nis_signal.png", fig)
fig

In [ ]:
fig = makie_samples(noise_samples)
save("compton_nis_noise.png", fig)
fig

In [ ]:
f_over_g = wi
mcint_nis = sum(f_over_g) / size(samples,2)
mcerror = sqrt(sum((f_over_g  .- mcint_nis).^2) / (size(samples,2)-1))
println("mc integral = $mcint_nis")
println("standard deviation = $mcerror")